In [47]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow
import re
import string
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
!git clone https://github.com/misbah4064/human-pose-estimation-opencv.git
%cd human-pose-estimation-opencv/

Cloning into 'human-pose-estimation-opencv'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), done.
/content/human-pose-estimation-opencv


In [7]:
def getShoulderLength(pointsInFrame):
  shoulderLength=0
  if (pointsInFrame[2]!=None):
    if pointsInFrame[5]!=None:
      shoulderLength=abs(pointsInFrame[2][0]-pointsInFrame[5][0])
    elif pointsInFrame[1]!=None:
      shoulderLength=abs(pointsInFrame[2][0]-pointsInFrame[1][0])*2
    elif pointsInFrame[20]!=None:
      shoulderLength=abs(pointsInFrame[2][0]-pointsInFrame[20][0])*2
  elif pointsInFrame[5]!=None:
    if pointsInFrame[1]!=None:
      shoulderLength=abs(pointsInFrame[5][0]-pointsInFrame[1][0])*2
    elif pointsInFrame[20]!=None:
      shoulderLength=abs(pointsInFrame[5][0]-pointsInFrame[20][0])*2
  return shoulderLength

In [8]:
def getEyeToAnkle(pointsInFrame):
  EyeToAnkleLength=0
  if (pointsInFrame[14]!=None):
    if pointsInFrame[10]!=None and pointsInFrame[13]!=None:
      if (pointsInFrame[10][1]>pointsInFrame[13][1]):
        EyeToAnkleLength=abs(pointsInFrame[10][1]-pointsInFrame[14][1])
      else:
        EyeToAnkleLength=abs(pointsInFrame[13][1]-pointsInFrame[14][1])
    elif pointsInFrame[10]!=None:
      EyeToAnkleLength=abs(pointsInFrame[10][1]-pointsInFrame[14][1])
    elif pointsInFrame[13]!=None:
      EyeToAnkleLength=abs(pointsInFrame[14][1]-pointsInFrame[13][1])
    elif pointsInFrame[29]!=None:
      EyeToAnkleLength=abs(pointsInFrame[14][1]-pointsInFrame[29][1])
    elif pointsInFrame[30]!=None:
      EyeToAnkleLength=abs(pointsInFrame[14][1]-pointsInFrame[30][1])+15
    elif pointsInFrame[24]!=None:
      EyeToAnkleLength=abs(pointsInFrame[24][1]-pointsInFrame[14][1])+15
  elif pointsInFrame[15]!=None:
    if pointsInFrame[10]!=None:
      EyeToAnkleLength=abs(pointsInFrame[10][1]-pointsInFrame[15][1])
    elif pointsInFrame[13]!=None:
      EyeToAnkleLength=abs(pointsInFrame[15][1]-pointsInFrame[13][1])
    elif pointsInFrame[29]!=None:
      EyeToAnkleLength=abs(pointsInFrame[15][1]-pointsInFrame[29][1])
    elif pointsInFrame[30]!=None:
      EyeToAnkleLength=abs(pointsInFrame[15][1]-pointsInFrame[30][1])+15
    elif pointsInFrame[24]!=None:
      EyeToAnkleLength=abs(pointsInFrame[24][1]-pointsInFrame[15][1])+15
  return EyeToAnkleLength

In [9]:
def getShirtLength(pointsInFrame):
  ShirtLength=0
  if (pointsInFrame[2]!=None):
    if pointsInFrame[9]!=None:
      if (pointsInFrame[9][1]<pointsInFrame[10][1]):
        ShirtLength=abs(pointsInFrame[2][1]-pointsInFrame[9][1])
    elif pointsInFrame[12]!=None:
      ShirtLength=abs(pointsInFrame[2][1]-pointsInFrame[12][1])
    elif pointsInFrame[22]!=None:
      ShirtLength=abs(pointsInFrame[22][1]-pointsInFrame[2][1])+20
    elif pointsInFrame[28]!=None:
      ShirtLength=abs(pointsInFrame[2][1]-pointsInFrame[28][1])+20
    elif pointsInFrame[8]!=None:
      ShirtLength=abs(pointsInFrame[2][1]-pointsInFrame[8][1])*1.5
    elif pointsInFrame[11]!=None:
      ShirtLength=abs(pointsInFrame[2][1]-pointsInFrame[11][1])*1.5
  elif pointsInFrame[5]!=None:
    if pointsInFrame[9]!=None:
      ShirtLength=abs(pointsInFrame[5][1]-pointsInFrame[9][1])
    elif pointsInFrame[12]!=None:
      ShirtLength=abs(pointsInFrame[5][1]-pointsInFrame[12][1])
    elif pointsInFrame[22]!=None:
      ShirtLength=abs(pointsInFrame[22][1]-pointsInFrame[5][1])+20
    elif pointsInFrame[28]!=None:
      ShirtLength=abs(pointsInFrame[5][1]-pointsInFrame[28][1])+20
    elif pointsInFrame[8]!=None:
      ShirtLength=abs(pointsInFrame[5][1]-pointsInFrame[8][1])*1.5
    elif pointsInFrame[11]!=None:
      ShirtLength=abs(pointsInFrame[5][1]-pointsInFrame[11][1])*1.5
  print(ShirtLength)
  return ShirtLength

In [12]:
def getShalwarLength(pointsInFrame):
  ShalwarLength=0
  if (pointsInFrame[8]!=None):
    if pointsInFrame[10]!=None:
      ShalwarLength=abs(pointsInFrame[8][1]-pointsInFrame[10][1])
    elif pointsInFrame[13]!=None:
      ShalwarLength=abs(pointsInFrame[8][1]-pointsInFrame[13][1])
    elif pointsInFrame[29]!=None:
      ShalwarLength=abs(pointsInFrame[8][1]-pointsInFrame[29][1])
    elif pointsInFrame[30]!=None:
      ShalwarLength=abs(pointsInFrame[8][1]-pointsInFrame[30][1])+20
    elif pointsInFrame[24]!=None:
      ShalwarLength=abs(pointsInFrame[8][1]-pointsInFrame[24][1])+20
  elif pointsInFrame[11]!=None:
    if pointsInFrame[10]!=None:
      ShalwarLength=abs(pointsInFrame[11][1]-pointsInFrame[10][1])
    elif pointsInFrame[13]!=None:
      ShalwarLength=abs(pointsInFrame[11][1]-pointsInFrame[13][1])
    elif pointsInFrame[29]!=None:
      ShalwarLength=abs(pointsInFrame[11][1]-pointsInFrame[29][1])
    elif pointsInFrame[30]!=None:
      ShalwarLength=abs(pointsInFrame[11][1]-pointsInFrame[30][1])+20
    elif pointsInFrame[24]!=None:
      ShalwarLength=abs(pointsInFrame[11][1]-pointsInFrame[24][1])+20
  return ShalwarLength

In [13]:
def getArmLength(pointsInFrame,frame):
  ArmLength=0
  if frame==0:
    if (pointsInFrame[2]!=None and pointsInFrame[4]!=None and pointsInFrame[2][0]>pointsInFrame[4][0]):
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[4][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[4][1])**2))
    elif (pointsInFrame[5]!=None and pointsInFrame[7]!=None and pointsInFrame[5][0]<pointsInFrame[7][0]):
      ArmLength=np.sqrt((abs(pointsInFrame[5][0]-pointsInFrame[7][0])**2)+(abs(pointsInFrame[5][1]-pointsInFrame[7][1])**2))
    elif pointsInFrame[2]!=None and pointsInFrame[7]!=None and pointsInFrame[2][0]>pointsInFrame[7][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[7][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[7][1])**2))
    elif pointsInFrame[5]!=None and pointsInFrame[4]!=None and pointsInFrame[5][0]<pointsInFrame[4][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[4][0]-pointsInFrame[5][0])**2)+(abs(pointsInFrame[4][1]-pointsInFrame[5][1])**2))
    elif pointsInFrame[2]!=None and pointsInFrame[3]!=None and pointsInFrame[2][0]>pointsInFrame[3][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[3][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[3][1])**2))*2
    elif pointsInFrame[5]!=None and pointsInFrame[6]!=None and pointsInFrame[5][0]<pointsInFrame[6][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[5][0]-pointsInFrame[6][0])**2)+(abs(pointsInFrame[5][1]-pointsInFrame[6][1])**2))*2
    elif pointsInFrame[2]!=None and pointsInFrame[6]!=None and pointsInFrame[2][0]>pointsInFrame[6][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[6][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[6][1])**2))*2
    elif pointsInFrame[5]!=None and pointsInFrame[3]!=None and pointsInFrame[5][0]<pointsInFrame[3][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[5][0]-pointsInFrame[3][0])**2)+(abs(pointsInFrame[5][1]-pointsInFrame[3][1])**2))*2
  elif frame==2:
    if (pointsInFrame[2]!=None and pointsInFrame[4]!=None and pointsInFrame[2][0]<pointsInFrame[4][0]):
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[4][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[4][1])**2))
    elif (pointsInFrame[5]!=None and pointsInFrame[7]!=None and pointsInFrame[5][0]>pointsInFrame[7][0]):
      ArmLength=np.sqrt((abs(pointsInFrame[5][0]-pointsInFrame[7][0])**2)+(abs(pointsInFrame[5][1]-pointsInFrame[7][1])**2))
    elif pointsInFrame[2]!=None and pointsInFrame[7]!=None and pointsInFrame[2][0]<pointsInFrame[7][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[7][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[7][1])**2))
    elif pointsInFrame[5]!=None and pointsInFrame[4]!=None and pointsInFrame[5][0]>pointsInFrame[4][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[4][0]-pointsInFrame[5][0])**2)+(abs(pointsInFrame[4][1]-pointsInFrame[5][1])**2))
    elif pointsInFrame[2]!=None and pointsInFrame[3]!=None and pointsInFrame[2][0]<pointsInFrame[3][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[3][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[3][1])**2))*2
    elif pointsInFrame[5]!=None and pointsInFrame[6]!=None and pointsInFrame[5][0]>pointsInFrame[6][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[5][0]-pointsInFrame[6][0])**2)+(abs(pointsInFrame[5][1]-pointsInFrame[6][1])**2))*2
    elif pointsInFrame[2]!=None and pointsInFrame[6]!=None and pointsInFrame[2][0]<pointsInFrame[6][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[2][0]-pointsInFrame[6][0])**2)+(abs(pointsInFrame[2][1]-pointsInFrame[6][1])**2))*2
    elif pointsInFrame[5]!=None and pointsInFrame[3]!=None and pointsInFrame[5][0]>pointsInFrame[3][0]:
      ArmLength=np.sqrt((abs(pointsInFrame[5][0]-pointsInFrame[3][0])**2)+(abs(pointsInFrame[5][1]-pointsInFrame[3][1])**2))*2
  return ArmLength

In [14]:
def getChestCircum(pointsInFrame):
  ChestCircum=0
  if (pointsInFrame[33]!=None and pointsInFrame[41]!=None):
    ChestCircum=abs(pointsInFrame[33][0]-pointsInFrame[41][0])*2.8
  elif pointsInFrame[33]!=None and pointsInFrame[20]!=None:
    ChestCircum=abs(pointsInFrame[33][0]-pointsInFrame[20][0])*4.8
  elif pointsInFrame[41]!=None and pointsInFrame[20]!=None:
    ChestCircum=abs(pointsInFrame[20][0]-pointsInFrame[41][0])*4.8
  elif pointsInFrame[41]!=None and pointsInFrame[26]!=None:
    ChestCircum=abs(pointsInFrame[26][0]-pointsInFrame[41][0])*4.8
  elif pointsInFrame[33]!=None and pointsInFrame[26]!=None:
    ChestCircum=abs(pointsInFrame[26][0]-pointsInFrame[33][0])*4.8
  return ChestCircum

In [16]:
def getWaistCircum(pointsInFrame):
  WaistCircum=0
  if (pointsInFrame[8]!=None and pointsInFrame[11]!=None):
    WaistCircum=abs(pointsInFrame[8][0]-pointsInFrame[11][0])*4.6
  elif pointsInFrame[8]!=None and pointsInFrame[28]!=None:
    WaistCircum=abs(pointsInFrame[8][0]-pointsInFrame[28][0])*4.6
  elif pointsInFrame[11]!=None and pointsInFrame[22]!=None:
    WaistCircum=abs(pointsInFrame[22][0]-pointsInFrame[11][0])*4.6
  return WaistCircum

In [17]:
def getBackCircum(pointsInFrame):
  BackCircum=0
  if (pointsInFrame[19]!=None and pointsInFrame[25]!=None):
    BackCircum=abs(pointsInFrame[19][0]-pointsInFrame[25][0])*4
  elif pointsInFrame[41]!=None and pointsInFrame[20]!=None:
    BackCircum=abs(pointsInFrame[41][0]-pointsInFrame[20][0])*4
  elif pointsInFrame[33]!=None and pointsInFrame[20]!=None:
    BackCircum=abs(pointsInFrame[20][0]-pointsInFrame[33][0])*4
  elif pointsInFrame[8]!=None and pointsInFrame[11]!=None:
    BackCircum=abs(pointsInFrame[8][0]-pointsInFrame[11][0])*4
  elif pointsInFrame[33]!=None and pointsInFrame[26]!=None:
    BackCircum=abs(pointsInFrame[26][0]-pointsInFrame[33][0])*4
  elif pointsInFrame[41]!=None and pointsInFrame[26]!=None:
    BackCircum=abs(pointsInFrame[41][0]-pointsInFrame[26][0])*4
  return BackCircum


In [48]:
def trained(frame):
    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                  "LShoulder": 5, "LElbow": 6, "LWrist": 7,"RHip": 8, "RKnee": 9,
                  "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
                  "LEye": 15, "REar": 16, "LEar": 17, "Background": 18,"RBack" : 19,"UnderNeck" :20,"Waste1" :21,"RThigh":22,"Waste2":23,"RAnkleAbove":24,
                  "LBack":25,"CentreUpChest":26,"Waste3":27,"LThigh":28,"LToLAnkle":29,"LAnkleAbove":30,"RToNeck":31,"Waste5":32,"RChest":33,"RElbowAbove":34,
                  "RElbowBelow":35,"RWristAbove":36,"RShoulderAbove":37,"Waste6":38,"LToNeck":39,"Waste7":40,"LChest":41,"LElbowAbove":42,"LElbowBelow":43,
                  "LWristAbove":44,"LShoulderAbove":45}
              
    POSE_PAIRS = [["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                  ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                  ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                  ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
                  ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"],["LEye","LAnkle"],
                  ["RShoulder", "RBack"],["UnderNeck","Neck"],["RKnee","RShoulder"],["LShoulder","LBack"],["Neck","CentreUpChest"],["LKnee","LShoulder"],
                  ["LChest","CentreUpChest"],["RHip","LHip"],["RChest","UnderNeck"],["LChest","UnderNeck"],["RChest","CentreUpChest"]]

    width = 368
    height = 368
    inWidth = width
    inHeight = height

    net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
    thr = 0.1
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :46, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 46 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    point1=[]
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x),int(y)) if conf > thr else None)
        point1.append((x,y) if conf > thr else None)
    print(len(points))
    for i in range(len(points)):
      print(i,points[i])
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]
        
        if points[idFrom] and points[idTo]:
             cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
             cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
             cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
    i=0
    for i in range(len(BODY_PARTS)):
      if( i==23 or i==21 or i==27 or i==32 or i==38 or i==40 or i==28 or i==30 or i==24):
        cv.ellipse(frame, points[i], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
      else:
        cv.ellipse(frame, points[i], (3, 3), 0, 0, 360, (0, 255, 255), cv.FILLED)
    t, _ = net.getPerfProfile()

    return point1,frame

In [49]:
class MeasurementModel:
  shoulderLength=""
  shirtLength=""
  shalwarLength=""
  armsLength=""
  chestCirum=""
  waistCircum=""
  backCircum=""
  def __init__(self):
    self.extractMeasurements() 
  def videoProcessing(self,path):
    vidcap = cv.VideoCapture(path)
    count = 0
    success = True
    fps = int(vidcap.get(cv.CAP_PROP_FPS))
    i=1
    im=0
    imagepaths=[]
    tempstr=""
    while success:
      success,image = vidcap.read()
      print('read a new frame:',success)
      if count%(i*fps) == 0 :
        if (len(image)<len(image[0])):
          image = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
        tempstr="/content/drive/My Drive/FYP/img"+str(im)+".jpg"
        cv.imwrite("/content/drive/My Drive/FYP/img"+str(im)+".jpg",image)
        imagepaths.append(tempstr)
        print('successful')
        im+=1
        i+=3
      count+=1
    return imagepaths
  def GetMeasurementsInPixels(self,pointsInFrame,CheckList,measurements,frame,health,gender):
    if (CheckList[0]==False):
      measurements[0]=getShoulderLength(pointsInFrame)
      if (measurements[0]>0):
        CheckList[0]=True
    if (CheckList[1]==False):
      measurements[1]=getEyeToAnkle(pointsInFrame)
      if (measurements[1]>0):
        CheckList[1]=True
    if (CheckList[2]==False):
      shirtLength=getShirtLength(pointsInFrame)
      if shirtLength>0 and shirtLength<measurements[1]:
        measurements[2]=shirtLength
        CheckList[2]=True
    if (CheckList[3]==False):
      measurements[3]=getShalwarLength(pointsInFrame)
      if measurements[3]>0:
        CheckList[3]=True
    if (CheckList[4]==False):
      measurements[4]=getArmLength(pointsInFrame,frame)
      if measurements[4]>0:
        CheckList[4]=True
    if (CheckList[5]==False):
      measurements[5]=getChestCircum(pointsInFrame)
      if measurements[5]>0:
        CheckList[5]=True
    if (CheckList[6]==False):
      measurements[6]=getWaistCircum(pointsInFrame)
      if measurements[6]>0:
        CheckList[6]=True
    if (CheckList[7]==False):
      measurements[7]=getBackCircum(pointsInFrame)
      if measurements[7]>0:
        CheckList[7]=True
    if frame>0:
      if getShoulderLength(pointsInFrame)>measurements[0] and getShoulderLength(pointsInFrame)<measurements[1]:
        measurements[0]=getShoulderLength(pointsInFrame)
      if getShalwarLength(pointsInFrame)>measurements[3] and getShirtLength(pointsInFrame)<measurements[1]:
        measurements[3]=getShalwarLength(pointsInFrame)
    if frame>=3:
      if (gender=="male"):
        if measurements[2]==0 or measurements[2]>(0.65*measurements[1]) or measurements[3]<(.6*measurements[1]):
          measurements[2]=measurements[1]*0.62
        if measurements[3]==0 or measurements[3]>(.58*measurements[1]) or measurements[3]<(.54*measurements[1]):
          measurements[3]=measurements[1]*0.57
        if measurements[0]==0 or measurements[0]>(.29*measurements[1]) or measurements[0]<(.25*measurements[1]):
          measurements[0]=measurements[1]*0.27
        if (health=="normal"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.59) or measurements[5]<(measurements[1]*0.55):
            measurements[5]=measurements[1]*.57
          if measurements[6]==0 or measurements[6]>(measurements[1]*.58) or measurements[6]<(measurements[1]*.55):
            measurements[6]=measurements[1]*.56
          if measurements[7]==0 or measurements[7]>(measurements[1]*.57) or measurements[7]<(measurements[1]*.52):
            measurements[7]=measurements[1]*.55
        if (health=="healthy"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.64) or measurements[5]<(measurements[1]*0.6):
            measurements[5]=measurements[1]*.62
          if measurements[6]==0 or measurements[6]>(measurements[1]*.67) or measurements[6]<(measurements[1]*.62):
            measurements[6]=measurements[1]*.64
          if measurements[7]==0 or measurements[7]>(measurements[1]*.61) or measurements[7]<(measurements[1]*.56):
            measurements[7]=measurements[1]*.55
        if (health=="slim"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.59) or measurements[5]<(measurements[1]*0.55):
            measurements[5]=measurements[1]*.57
          if measurements[6]==0 or measurements[6]>(measurements[1]*.55) or measurements[6]<(measurements[1]*.52):
            measurements[6]=measurements[1]*.54
          if measurements[7]==0 or measurements[7]>(measurements[1]*.58) or measurements[7]<(measurements[1]*.54):
            measurements[7]=measurements[1]*.52
        if (health=="bulky"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.60) or measurements[5]<(measurements[1]*0.65):
            measurements[5]=measurements[1]*.63
          if measurements[6]==0 or measurements[6]>(measurements[1]*.7) or measurements[6]<(measurements[1]*.57):
            measurements[6]=measurements[1]*.69
          if measurements[7]==0 or measurements[7]>(measurements[1]*.65) or measurements[7]<(measurements[1]*.62):
            measurements[7]=measurements[1]*.64
      if (gender=="female"):
        if measurements[2]==0 or measurements[2]>(0.59*measurements[1]) or measurements[3]<(.55*measurements[1]):
          measurements[2]=measurements[1]*0.57
        if measurements[3]==0 or measurements[3]>(.57*measurements[1]) or measurements[3]<(.52*measurements[1]):
          measurements[3]=measurements[1]*0.55
        if measurements[0]==0 or measurements[0]>(.27*measurements[1]) or measurements[0]<(.24*measurements[1]):
          measurements[0]=measurements[1]*0.25
        if measurements[4]==0 or measurements[4]>(.32*measurements[1]) or measurements[0]<(.30*measurements[1]):
          measurements[4]=measurements[1]*.32
        if (health=="normal"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.58) or measurements[5]<(measurements[1]*0.55):
            measurements[5]=measurements[1]*.56
          if measurements[6]==0 or measurements[6]>(measurements[1]*.56) or measurements[6]<(measurements[1]*.52):
            measurements[6]=measurements[1]*.54
          if measurements[6]==0 or measurements[7]>(measurements[1]*.52) or measurements[6]<(measurements[1]*.49):
            measurements[6]=measurements[1]*.51
        if (health=="healthy"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.60) or measurements[5]<(measurements[1]*0.58):
            measurements[5]=measurements[1]*.59
          if measurements[6]==0 or measurements[6]>(measurements[1]*.56) or measurements[6]<(measurements[1]*.52):
            measurements[6]=measurements[1]*.54
          if measurements[7]==0 or measurements[7]>(measurements[1]*.54) or measurements[7]<(measurements[1]*.5):
            measurements[7]=measurements[1]*.52
        if (health=="slim"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.58) or measurements[5]<(measurements[1]*0.54):
            measurements[5]=measurements[1]*.56
          if measurements[6]==0 or measurements[6]>(measurements[1]*.54) or measurements[6]<(measurements[1]*.50):
            measurements[6]=measurements[1]*.52
          if measurements[7]==0 or measurements[7]>(measurements[1]*.52) or measurements[7]<(measurements[1]*.48):
            measurements[7]=measurements[1]*.5
        if (health=="bulky"):
          if measurements[5]==0 or measurements[5]>(measurements[1]*.64) or measurements[5]<(measurements[1]*0.6):
            measurements[5]=measurements[1]*.62
          if measurements[6]==0 or measurements[6]>(measurements[1]*.7) or measurements[6]<(measurements[1]*.67):
            measurements[6]=measurements[1]*.69
          if measurements[7]==0 or measurements[7]>(measurements[1]*.62) or measurements[7]<(measurements[1]*.59):
            measurements[7]=measurements[1]*.61
    print(measurements)
    return CheckList,measurements 
  def pixelToInches(self,measurements,height):
    height=str (height)
    heightarr=re.split("\.",height)
    height=(int(heightarr[0])*12)+int(heightarr[1])
    print(height)
    pixelsPerInch=measurements[1]/(height-4)
    originalmeasurements=[]
    self.shoulderLength=measurements[0]/pixelsPerInch
    self.shirtLength=measurements[2]/pixelsPerInch
    self.shalwarLength=measurements[3]/pixelsPerInch
    self.armsLength=measurements[4]/pixelsPerInch
    self.chestCirum=measurements[5]/pixelsPerInch
    self.waistCircum=measurements[6]/pixelsPerInch
    self.backCircum=measurements[7]/pixelsPerInch
  def extractMeasurements(self):
    height=input("Enter height in feet:")
    health=input("Are you slim,normal,healthy,bulky:")
    gender=input("Enter your gender:")
    path='/content/drive/My Drive/FYP/3.mp4'  #video path
    imagepaths=self.videoProcessing(path)
    CheckList=[False,False,False,False,False,False,False,False] 
    measurements=[0]*len(CheckList)
    for i in range(len(imagepaths)):
      input1 = cv.imread(imagepaths[i])
      frame=i
      pointsPerFrame,output =trained(input1)
      inp=cv2.imread(imagepaths[0])
      points,output=trained(inp)
      #cv2_imshow(output)
      CheckList,measurements=self.GetMeasurementsInPixels(pointsPerFrame,CheckList,measurements,frame,health,gender)
      print(measurements)
    if (measurements[1]<=0):
      print("kindly make a video again")
    self.pixelToInches(measurements,height)
    print("Shoulder to Shoulder",self.shoulderLength)
    print("Shirt Length",self.shirtLength)
    print("Shalwar Length",self.shalwarLength)
    print("Arm Length",self.armsLength)
    print("Chest",self.chestCirum)
    print("Waist",self.waistCircum)
    print("UnderChest(Back)",self.backCircum)

In [50]:
z=MeasurementModel()

Enter height in feet:5.8
Are you slim,normal,healthy,bulky:normal
Enter your gender:male
read a new frame: True
successful
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
read a new frame: True
rea

**Original Measurements**

awais measurements:
sh to sh = 17.3
shirt length =39.5
shalwar length = 36
sh to wrist =20
chest = 40
waist=41
Back= 35

Feroz measurements:
sh to sh =17.5
shirt length =40
shalwar length=37
sh to wrist= 23.5
chest = 37
waist= 36.5
back=35.7

zeemal measurements:
sh to sh=13.7
Shirt Length=31.3
Shalwar Length=30.3
Arm Length=17.6
Chest = 34
waist = 38
Back=33.5

for checking the visualisation

In [ ]:
!pip install tf-bodypix[all]

In [ ]:
!python -m tf_bodypix \
    draw-mask \
    --source '/content/drive/My Drive/FYP/img1.jpg' \
    --output '/content/drive/My Drive/FYP/out3.jpg' \
    --threshold=0.5 \
    --colored

   8192/Unknown - 0s 0us/stepDownloading data from https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/float/050/group1-shard1of1.bin
   8192/Unknown - 0s 0us/stepINFO:tf_bodypix.cli:loading model: '/root/.keras/tf-bodypix/8ba301b16e59fd7bda330880a9d70e58-https-storage-googleapis-com-tfjs-models-savedmodel-bodypix-mobilenet-float-050-model-stride16' (downloaded from 'https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/float/050/model-stride16.json')
INFO:tf_bodypix.sink:writing image to: '/content/drive/My Drive/FYP/out3.jpg'


In [ ]:
!python -m tf_bodypix \
    draw-mask \
    --source '/content/drive/My Drive/FYP/imgnext.jpg' \
    --output '/content/drive/My Drive/FYP/outnew.jpg' \
    --threshold=0.75 \
    --parts 'torso_front' 'torso_back'\
    --colored

INFO:tf_bodypix.cli:loading model: '/root/.keras/tf-bodypix/8ba301b16e59fd7bda330880a9d70e58-https-storage-googleapis-com-tfjs-models-savedmodel-bodypix-mobilenet-float-050-model-stride16' (downloaded from 'https://storage.googleapis.com/tfjs-models/savedmodel/bodypix/mobilenet/float/050/model-stride16.json')
INFO:tf_bodypix.sink:writing image to: '/content/drive/My Drive/FYP/outnew.jpg'
